In [1]:
import sys
from pathlib import Path
from datetime import datetime

# Get the parent directory of current notebook dir, which is the repo root
repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root))

# Verify the path is correct:
print(f"Added to sys.path: {repo_root}")

import pandas as pd


# Now import your module
from scripts.utils.config import get_config


Added to sys.path: c:\Users\n740789\Documents\clarity_data_quality_controls


In [2]:
config = get_config("explore-sustainalytics-nasdaq-meetrics", auto_date=False, fixed_date="202505")
logger = config["logger"]
sutainalytics_path = config["paths"]["SUSTAINALYTICS_DATA_PATH"]
nasdaq_path = config["paths"]["NASDAQ_DATA_PATH"]

In [3]:
nasdaq = pd.read_excel(nasdaq_path, skiprows=3)
sutainalytics = pd.read_excel(sutainalytics_path, skiprows=3)

c:\Users\n740789\Documents\clarity_data_quality_controls\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\n740789\Documents\clarity_data_quality_controls\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
nasdaq.head()

,cusip,issuer_id,security_description,nasdaq_bnd
0,SNT-WORLD,NaN,NaN,NaN
1,Cash and cash equiv,NaN,NaN,NaN
2,Cash,NaN,NaN,NaN
3,Dollar Bloc,NaN,NaN,NaN
4,AUD_CCASH,C05702,AUD CASH(Alpha Committed),NaN


In [8]:
sutainalytics.head()

,issuer_id,security_description,highest_controversy_level_answer_category,overall_global_compact_compliance_status
0,SNT-WORLD,NaN,2.023855,NaN
1,Cash and cash equiv,NaN,2.711829,NaN
2,Cash,NaN,NaN,NaN
3,Dollar Bloc,NaN,NaN,NaN
4,NaN,CASH,NaN,NaN


In [6]:
def clean_cols(col_name:str)->str:
    """
    Cleans the column names by removing unwanted characters and spaces.
    """
    # Remove unwanted characters and spaces
    col_name = col_name.replace("(Wt Avg-PORT Delta NMV)","").replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_").lower()
    return col_name

In [7]:
nasdaq.columns = [clean_cols(col) for col in nasdaq.columns]
sutainalytics.columns = [clean_cols(col) for col in sutainalytics.columns]

In [10]:
def clean_df(df: pd.DataFrame, target_cols:list[str]) -> pd.DataFrame:
    """
    Drop rows where all the target columns are empty (NaN).
    """
    # Create a mask for rows where all target columns are NaN
    mask = df[target_cols].isna().all(axis=1)
    # Keep only rows where not all target columns are NaN
    return df[~mask]

In [11]:
nasdaq_clean = clean_df(nasdaq, ["cusip", "nasdaq_bnd"])

In [ ]:
sutainalytics_clean = clean_df(sutainalytics, ["issuer_id", "highest_controversy_level_answer_category"])